<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo-52/blob/main/Reto%20-%20Entrega%201_prueba1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> # **Reto - Entrega 1**
>### **Maestría en Inteligencia Artificial Aplicada**
>### **Curso: Inteligencia Artificial y Aprendizaje Automático**
>### **Tecnológico de Monterrey**
>### **Prof: María de la Paz Rico Fernández**

### **Genaro Ramos Higuera - A00351269**
### **Gerardo Aaron Castañeda Jaramillo - A01137646**

# **Limpieza, análisis, visualización y agrupamiento.**

In [21]:
import pandas as pd
import numpy as np
from scipy import stats

import matplotlib.pyplot as plt 
import seaborn as sns 

import requests
import zipfile as zf
from io import BytesIO

In [22]:
def zip_to_df(url, filename, separator):

  print('Downloading started')
  # Split URL to get the file name
  req = requests.get(url)

  foldername = url.split('/')[-1].split('.')[-2]
  print('Downloading Completed') 

  # extracting the zip file contents
  zipfile= zf.ZipFile(BytesIO(req.content))
  zipfile.extractall('/content/sample_data/' + foldername)
  df = pd.read_csv("/content/sample_data/" + foldername + '/Datos_de_calidad_del_agua_2020/' + filename,sep=separator,encoding='latin-1')

  return df

## **Aguas subterraneas**

In [118]:
url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

df_asub = zip_to_df(url,'Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_subterraneas_2020.csv',',')
df_asub.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,ACUIFERO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DUR,CUMPLE_CON_CF,CUMPLE_CON_NO3,CUMPLE_CON_AS,CUMPLE_CON_CD,CUMPLE_CON_CR,CUMPLE_CON_HG,CUMPLE_CON_PB,CUMPLE_CON_MN,CUMPLE_CON_FE
0,DLAGU6,POZO SAN GIL,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,ASIENTOS,VALLE DE CHICALOTE,POZO,-102.02210,22.20887,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
1,DLAGU6516,POZO R013 CAÑADA HONDA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,AGUASCALIENTES,VALLE DE CHICALOTE,POZO,-102.20075,21.99958,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
2,DLAGU7,POZO COSIO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,COSIO,VALLE DE AGUASCALIENTES,POZO,-102.28801,22.36685,2020,...,SI,SI,SI,NO,SI,SI,SI,SI,SI,SI
3,DLAGU9,POZO EL SALITRILLO,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,VALLE DE AGUASCALIENTES,POZO,-102.29449,22.18435,2020,...,SI,SI,SI,SI,SI,SI,SI,SI,SI,SI
4,DLBAJ107,RANCHO EL TECOLOTE,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,POZO,-110.24480,23.45138,2020,...,SI,SI,NO,SI,SI,SI,SI,SI,SI,SI


In [128]:
asub = df_asub.copy()
asub.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 57 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   CLAVE                 1068 non-null   object 
 1   SITIO                 1068 non-null   object 
 2   ORGANISMO_DE_CUENCA   1068 non-null   object 
 3   ESTADO                1068 non-null   object 
 4   MUNICIPIO             1068 non-null   object 
 5   ACUIFERO              1068 non-null   object 
 6   SUBTIPO               1068 non-null   object 
 7   LONGITUD              1068 non-null   float64
 8   LATITUD               1068 non-null   float64
 9   PERIODO               1068 non-null   int64  
 10  ALC_mg/L              1064 non-null   float64
 11  CALIDAD_ALC           1064 non-null   object 
 12  CONDUCT_mS/cm         1062 non-null   float64
 13  CALIDAD_CONDUC        1062 non-null   object 
 14  SDT_mg/L              0 non-null      float64
 15  SDT_M_mg/L           

Donde:
* 0 - CLAVE: es como un ID
* 1 - SITIO: es categorica sobre ubicacion
* 2 - ORGANISMO_DE_CUENCA: categorica que identifica al responsables de administrar y preservar las aguas nacionales en cada una de las trece regiones hidrológico-administrativas en que se ha dividido el país
* 3 - ESTADO: categorica sobre el estado en que esta ubicado
* 4 - MUNICIPIO: categorica sobre el municipio en que esta ubicado
* 5 - ACUIFERO: categorica sobre el acuifero al que pertenece
* 6 - SUBTIPO: categorica sobre el tipo de yacimiento al que pertenece
* 7 - LONGITUD: numerica y parte de coordenada geografica
* 8 - LATITUD: numerica y parte de coordenada geografica
* 9 - PERIODO: periodo de recoleccion de datos (todos son 2020)
* 10 - ALC_mg/L: numerica de calidad del agua para alcalinidad
* 11 - CALIDAD_ALC: categorica de calidad del agua para alcalinidad
* 12 - CONDUCT_mS/cm: numerica de calidad del agua para conductividad
* 13 - CALIDAD_CONDUC: categorica de calidad del agua para conductividad
* 14 - SDT_mg/L: numerica de calidad del agua para SOLIDOS DISUELTOS TOTALES (numerica vacia)
* 15 - SDT_M_mg/L: numerica de calidad del agua para solidos disueltos totales
* 16 - CALIDAD_SDT_ra: categorica de calidad del agua para solidos disueltos totales - riego agricola
* 17 - CALIDAD_SDT_salin: categorica de calidad del agua para solidos disueltos totales - SALINIZACION
* 18 - FLUORUROS_mg/L: numerica de calidad del agua para FLUORUROS
* 19 - CALIDAD_FLUO: categorica de calidad del agua para FLUORUROS
* 20 - DUR_mg/L: numerica de calidad del agua para DUREZA
* 21 - CALIDAD_DUR: categorica de calidad del agua para DUREZA
* 22 - COLI_FEC_NMP/100_mL: numerica de calidad del agua para COLIFORMES FECALES
* 23 - CALIDAD_COLI_FEC: categorica de calidad del agua para COLIFORMES FECALES
* 24 - N_NO3_mg/L: numerica de calidad del agua para NITROGENO DE NITRATOS 
* 25 - CALIDAD_N_NO3: categorica de calidad del agua para NITROGENO DE NITRATOS 
* 26 - AS_TOT_mg/L: numerica de calidad del agua para ARSENICO
* 27 - CALIDAD_AS: categorica de calidad del agua para ARSENICO
* 28 - CD_TOT_mg/L: numerica de calidad del agua para CADMIO
* 29 - CALIDAD_CD: categorica de calidad del agua para CADMIO
* 30 - CR_TOT_mg/L: numerica de calidad del agua para CROMO
* 31 - CALIDAD_CR: categorica de calidad del agua para CROMO
* 32 - HG_TOT_mg/L: numerica de calidad del agua para MERCURIO
* 33 - CALIDAD_HG: categorica de calidad del agua para MERCURIO
* 34 - PB_TOT_mg/L: numerica de calidad del agua para PLOMO
* 35 - CALIDAD_PB: categorica de calidad del agua para PLOMO
* 36 - MN_TOT_mg/L: numerica de calidad del agua para MANGANESO
* 37 - CALIDAD_MN: categorica de calidad del agua para MANGANESO
* 38 - FE_TOT_mg/L: numerica de calidad del agua para MANGANESO
* 39 - CALIDAD_FE: categorica de calidad del agua para MANGANESO
* 40 - SEMAFORO: categorica sobre si el agua es potable o no
* 41 - CONTAMINANTES: string de que contaminantes tiene, dependiente de las variables 10-39
* 42 - CUMPLE_CON_ALC: binaria sobre si cumple criterio de CALIDAD_ALC
* 43 - CUMPLE_CON_COND: binaria sobre si cumple criterio de CALIDAD_COND
* 44 - CUMPLE_CON_SDT_ra: binaria sobre si cumple criterio de CALIDAD_STD_ra
* 45 - CUMPLE_CON_SDT_salin: binaria sobre si cumple criterio de CALIDAD_STD_salin
* 46 - CUMPLE_CON_FLUO: binaria sobre si cumple criterio de CALIDAD_FLUO
* 47 - CUMPLE_CON_DUR: binaria sobre si cumple criterio de CALIDAD_DUR
* 48 - CUMPLE_CON_CF: binaria sobre si cumple criterio de CALIDAD_CF
* 49 - CUMPLE_CON_NO3: binaria sobre si cumple criterio de CALIDAD_NO3
* 50 - CUMPLE_CON_AS: binaria sobre si cumple criterio de CALIDAD_AS
* 51 - CUMPLE_CON_CD: binaria sobre si cumple criterio de CALIDAD_CD
* 52 - CUMPLE_CON_CR: binaria sobre si cumple criterio de CALIDAD_CR
* 53 - CUMPLE_CON_HG: binaria sobre si cumple criterio de CALIDAD_HG
* 54 - CUMPLE_CON_PB: binaria sobre si cumple criterio de CALIDAD_PB
* 55 - CUMPLE_CON_MN: binaria sobre si cumple criterio de CALIDAD_MN
* 56 - CUMPLE_CON_FE: binaria sobre si cumple criterio de CALIDAD_FE



De entrada observamos que 'SDT_mg/L ' no tiene valores, asi que eliminamos la columna del dataframe:

In [129]:
asub.drop('SDT_mg/L', inplace=True, axis=1)

De igual manera observamos que existen dos valores de calidad para solidos disueltos totales, siendo una de ellas para uso agricola. Y solo nos interesa la de salinizacion, ya que es la que nos determina si es potable o no. Asi que la eliminamos lo referente a ella:

In [130]:
asub.drop('CALIDAD_SDT_ra', inplace=True, axis=1)
asub.drop('CUMPLE_CON_SDT_ra', inplace=True, axis=1)

Remplazamos algunos valores de nombres de columnas para que todas tengan el mismo formato:

In [131]:
asub.rename(columns = {'COLI_FEC_NMP/100_mL':'CF_NMP/100_mL','CALIDAD_COLI_FEC':'CALIDAD_CF',
                       'N_NO3_mg/L':'NNO3_mg/L','CALIDAD_N_NO3':'CALIDAD_NNO3','CUMPLE_CON_NO3':'CUMPLE_CON_NNO3',
                       'FLUORUROS_mg/L':'FLUO_mg/L',
                       'CONDUCT_mS/cm':'COND_mS/cm','CALIDAD_CONDUC':'CALIDAD_COND',
                       'CALIDAD_SDT_salin':'CALIDAD_SDT','CUMPLE_CON_SDT_salin':'CUMPLE_CON_SDT'
                       }, inplace = True)

Separamos variables por sus tipos:

In [133]:
#definimos variables numéricas #16
num_nom = ['LONGITUD','LATITUD','ALC_mg/L','COND_mS/cm','SDT_M_mg/L','FLUO_mg/L','DUR_mg/L','CF_NMP/100_mL','NNO3_mg/L','AS_TOT_mg/L',
           'CD_TOT_mg/L','CR_TOT_mg/L','HG_TOT_mg/L','PB_TOT_mg/L','MN_TOT_mg/L','FE_TOT_mg/L']
#definimos variables categóricas #22
cat_nom = ['SITIO','ORGANISMO_DE_CUENCA','ESTADO','MUNICIPIO','ACUIFERO','SUBTIPO','CALIDAD_COND','CALIDAD_ALC','CALIDAD_SDT','CALIDAD_FLUO','CALIDAD_DUR',
           'CALIDAD_CF','CALIDAD_NNO3','CALIDAD_AS','CALIDAD_CD','CALIDAD_CR','CALIDAD_HG','CALIDAD_PB','CALIDAD_MN','CALIDAD_FE','CONTAMINANTES']
#definimos variables ordinales #1
ord_nom = ['PERIODO']
#definimos variables binarias #14
bin_nom = ['CUMPLE_CON_ALC','CUMPLE_CON_COND','CUMPLE_CON_SDT','CUMPLE_CON_FLUO','CUMPLE_CON_DUR','CUMPLE_CON_CF','CUMPLE_CON_NNO3','CUMPLE_CON_AS','CUMPLE_CON_CD',
           'CUMPLE_CON_CR','CUMPLE_CON_HG','CUMPLE_CON_PB','CUMPLE_CON_MN','CUMPLE_CON_FE']
#VARIABLE CATEGORICA DE SALIDA Y  #1
y_nom = ['SEMAFORO']

Y comenzamos a explorar para limpiar:

* Variables numericas:

In [134]:
asub[num_nom].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LONGITUD       1068 non-null   float64
 1   LATITUD        1068 non-null   float64
 2   ALC_mg/L       1064 non-null   float64
 3   COND_mS/cm     1062 non-null   float64
 4   SDT_M_mg/L     1066 non-null   object 
 5   FLUO_mg/L      1068 non-null   object 
 6   DUR_mg/L       1067 non-null   object 
 7   CF_NMP/100_mL  1068 non-null   object 
 8   NNO3_mg/L      1067 non-null   object 
 9   AS_TOT_mg/L    1068 non-null   object 
 10  CD_TOT_mg/L    1068 non-null   object 
 11  CR_TOT_mg/L    1068 non-null   object 
 12  HG_TOT_mg/L    1068 non-null   object 
 13  PB_TOT_mg/L    1068 non-null   object 
 14  MN_TOT_mg/L    1068 non-null   object 
 15  FE_TOT_mg/L    1068 non-null   object 
dtypes: float64(4), object(12)
memory usage: 133.6+ KB


Observamoms que no todas nos arrojan como Dtype float, lo cual nos impide utilizar el metodo de describe(), asi que las comenzamos a analizar una por una para determinar que sucede:

In [115]:
14/1068

0.013108614232209739

Longitud y latitud no contienen valores nulos y ya estan en float64, asi que se quedan asi. Sin embargo, observamos que para las demas variables, algunas si tienen valores nulos. Estas variables estan relacionadas con su respectiva variable categorica de calidad, asi como su variable binaria sobre si cumple o no con el criterio de calidad. Igualmente, si alguna variable binaria no cumple, la variable de CONTAMINANTES lo refleja. Por lo que observaremos sus valores faltantes junto con sus variables relacionadas. 

In [135]:
temp_num_nom = num_nom.copy()
temp_num_nom.remove('LONGITUD')
temp_num_nom.remove('LATITUD')

for col in temp_num_nom:
  new_cols = list()
  if asub[col].isna().sum() > 0:
    contaminant = col.split('_', 1)[0]
    new_cols.append(col)
    new_cols.append('CALIDAD_'+contaminant)
    new_cols.append('CUMPLE_CON_'+contaminant)
    new_cols.append('CONTAMINANTES')
    new_cols.append('SEMAFORO')
    indx_na = asub[asub[col].isnull()].index.tolist()
    temp_df = pd.DataFrame(columns=new_cols)
    if len(indx_na) > 0:
      for indx in indx_na:
        temp_df = temp_df.append(asub[new_cols].iloc[indx], ignore_index = True)
      print(str(len(indx_na)),'NA values in numeric variable:',col)
      display(temp_df)

4 NA values in numeric variable: ALC_mg/L


,ALC_mg/L,CALIDAD_ALC,CUMPLE_CON_ALC,CONTAMINANTES,SEMAFORO
0,NaN,NaN,ND,"NO3,",Rojo
1,NaN,NaN,ND,NaN,Verde
2,NaN,NaN,ND,NaN,Verde
3,NaN,NaN,ND,"FLUO,",Rojo


6 NA values in numeric variable: COND_mS/cm


,COND_mS/cm,CALIDAD_COND,CUMPLE_CON_COND,CONTAMINANTES,SEMAFORO
0,NaN,NaN,ND,"AS,",Rojo
1,NaN,NaN,ND,NaN,Verde
2,NaN,NaN,ND,"ALC,FLUO,AS,",Rojo
3,NaN,NaN,ND,"ALC,SDT_ra,SDT_salin,DT,MN,FE,",Amarillo
4,NaN,NaN,ND,NaN,Verde
5,NaN,NaN,ND,NaN,Verde


2 NA values in numeric variable: SDT_M_mg/L


,SDT_M_mg/L,CALIDAD_SDT,CUMPLE_CON_SDT,CONTAMINANTES,SEMAFORO
0,NaN,NaN,ND,"NO3,",Rojo
1,NaN,NaN,ND,"FE,",Amarillo


1 NA values in numeric variable: DUR_mg/L


,DUR_mg/L,CALIDAD_DUR,CUMPLE_CON_DUR,CONTAMINANTES,SEMAFORO
0,NaN,NaN,ND,NaN,Verde


1 NA values in numeric variable: NNO3_mg/L


,NNO3_mg/L,CALIDAD_NNO3,CUMPLE_CON_NNO3,CONTAMINANTES,SEMAFORO
0,NaN,NaN,ND,NaN,Verde


Donde encontramos 5 tipos de contaminantes que contienen datos nulos. Y efectivamente, comprobamos que si falta un dato en la cantidad del contaminante, tambien falta en su variable de calidad, no lo menciona en la variable de contaminantes, y no tiene efecto alguno en la variable de salida. Asi mismo, la columna de contaminantes solo es un indicador de que contaminante no cumple con su criterio de agua potable. 

Si cada tipo de variable numerica/categorica/binaria representa lo mismo. Entonces nos quedamos con las variables binarias, y obtenemos nueva dataframe:

## **Aguas superficiales**

In [13]:
url = 'http://201.116.60.46/Datos_de_calidad_del_agua_de_5000_sitios_de_monitoreo.zip'

df_asup = zip_to_df(url,'Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_superficiales_2020.csv',',')
df_asup.head()

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,CUENCA,CUERPO DE AGUA,TIPO,SUBTIPO,LONGITUD,...,CONTAMINANTES,CUMPLE_CON_DBO,CUMPLE_CON_DQO,CUMPLE_CON_SST,CUMPLE_CON_CF,CUMPLE_CON_E_COLI,CUMPLE_CON_ENTEROC,CUMPLE_CON_OD,CUMPLE_CON_TOX,GRUPO
0,DLAGU8,PRESA EL SAUCILLO 100M AGUAS ARRIBA DE LA CORTINA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,RIO SAN PEDRO,PRESA EL SAUCILLO,LENTICO,PRESA,-102.33911,...,"DQO,CF,",SI,NO,SI,NO,SI,ND,SI,SI,LENTICO
1,DLBAJ100,"LOS CABOS SEG 22, 2 ISA10B",PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN JOSE DEL CABO,OCEANO PACIFICO,COSTERO,OCEANO-MAR,-109.84290,...,NaN,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO
2,DLBAJ101,"LOS CABOS SEG 22, 1 ISA10B",PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,OCEANO PACIFICO,COSTERO,OCEANO-MAR,-109.86442,...,NaN,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO
3,DLBAJ102,LOS CABOS 3,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.88604,...,NaN,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO
4,DLBAJ103,LOS CABOS 1,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.89657,...,NaN,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO


In [16]:
df_asup.shape

(4141, 55)